# parsing raw spike times from hdf5 (runtime ~7 days)

In [1]:
#automate the boring stuff
from lib.getterdone import *
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import json,os

In [5]:
#search_for_filename
input_file_name=search_for_file()
#manual data has an ugly dtype
# input_file_name='/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/SRT_CollabData/SortedData_Manual/catTempPopStorage_full.mat'
#early autogen data
#Archie
# input_file_name='/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/SRT_CollabData/SortedData_AutoGen/Archie_SRT_Set226_subset1_201020_142509_UnitStorage.mat'
#

In [4]:
#extract all spike times for all trials in .mat
# field_lst=['chanNum', 'featSpace', 'spikeInds', 'spikeTimes', 'trialFR', 'trialPSTH', 'trialSpks', 'wvfrms']
with h5py.File(input_file_name, 'r') as f:
    obj=f['SRTunitStorage']['trialSpks']
    num_neurons=obj.shape[0]
    neuron_id_lst=sorted(range(num_neurons))
    for trial_id in sorted(range(53,452)):
        spikes_lst=[]
        for nid in neuron_id_lst:
            ref=obj[(nid)][0]
            name = h5py.h5r.get_name(ref, f.id)
            trials = np.array(f[(name)]).flatten();#print(name) #not sure if trials covers trials or neurons... its length is ~450.
            ref=trials[trial_id]
            name = h5py.h5r.get_name(ref, f.id)
            spikes=np.array(f[(name)]);#print(f"{nid}:{name},")
            spikes_lst.append(spikes)

        dict_spike_times={}
        for nid,spikes in zip(neuron_id_lst,spikes_lst):
            dict_spike_times[nid]=list(spikes.flatten().astype(np.float64))

        # the json file where the output must be stored 
#         os.chdir(f"{nb_dir}/Data/Archie")
        os.chdir(f"{nb_dir}/Data/Archie")
        out_fn=f"spike_times_trial_{trial_id}.json"
        out_file = open(out_fn, "w") 
        json.dump(dict_spike_times, out_file, indent = 6) 
        out_file.close() 

print(os.path.abspath(out_fn))

/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/Archie/spike_times_trial_451.json


- % SRTunitStorage – 1 x n Neurons struct
- %     % chanNum – channel ID number Neuron was located on
- %     % spikeInds – Index of Spikes from that neuron (in 30kHz recording time)
- %     % spikeTimes – Index of Spikes converted to Time (sec), relative to recording start
- %     % trialSpks – SpikeTimes for each individual Trial, Time relative to Stimlus onset (sec)
- %     % trialFR – 1 x nTrials cell array 
- %     % 	 Cell is 49 bins, ea the instantaneuous firing rate (Hz) relative to stimulus onset
- %     % 	I believe they are 100msec bins, -2sec to +3 sec
- %     % trialPSTH – Histogram of firing relative to Stimulus onset per Trial
- %     % 	100ms bins, raw n spikes (not converted to Hz) 

In [ ]:
#TODO: import mike's .mat file
#Extract the fields for one trial
import h5py
trial_id=-1
field_lst=['chanNum', 'featSpace', 'spikeInds', 'spikeTimes', 'trialFR', 'trialPSTH', 'trialSpks', 'wvfrms']
with h5py.File(input_file_name, 'r') as f:
#     print(f.keys())
#     print(mat:=f['SRTunitStorage'].keys())#.fletcher32)#.items())#__dir__())
#     print(obj:=f['SRTunitStorage']['spikeTimes'])

    trial=f['SRTunitStorage']['spikeTimes'][(trial_id)][0]
    name = h5py.h5r.get_name(trial, f.id)
    data = np.array(f[(name)])
    ref=f['SRTunitStorage']['chanNum'][(trial_id)][0]
    name = h5py.h5r.get_name(ref, f.id)
    chnlno = int(np.array(f[(name)]))
    
    obj=f['SRTunitStorage']['featSpace'][(trial_id)]
    print(obj.shape)
    ref=obj[0]
    name = h5py.h5r.get_name(ref, f.id)
    featSpace = np.array(f[(name)])
    
    obj=f['SRTunitStorage']['spikeInds'][(trial_id)]
    print(obj.shape)
    ref=obj[0]
    name = h5py.h5r.get_name(ref, f.id)
    spikeInds = np.array(f[(name)])
    
    obj=f['SRTunitStorage']['trialSpks'][(trial_id)]
    print(obj.shape)
    ref=obj[0]
    name = h5py.h5r.get_name(ref, f.id)
    trialSpks = np.array(f[(name)])
    
#     chnlno_lst=[]
#     for trial_id in range(195):
#         ref=f['SRTunitStorage']['chanNum'][(trial_id)][0]
#         name = h5py.h5r.get_name(ref, f.id)
#         chnlno = int(np.array(f[(name)]))
#         chnlno_lst.append(chnlno)

# (deprecated) parse raw spike times from csv (runtime <2 mintues)
this schema is described in doc/schema.docx

In [2]:
from lib.getterdone import *

In [16]:
#find the raw data files of interest
data_dir=f"{nb_dir}/Data/Archie"
# data_dir=f"{nb_dir}/Data/Buck"

# TODO: parse the following... the method needs editing to notice the right files
# data_dir=f"{nb_dir}/Data/Baloo"
# data_dir=f"{nb_dir}/Data/Hades"
# data_dir=f"{nb_dir}/Data/Karl"
os.chdir(data_dir)
fn_lst=sorted([fn for fn in os.listdir() if fn[-4:]=='.csv'])
trgt1='trialTimes.csv'
trgt2='trialData.csv'
trgt3='ChannelNums.csv'
trgt4='SpikeTimes.csv'
input_fn_lst=[]
setnum_lst=[]
trgt=trgt4
for fn in fn_lst:
    if fn.find(trgt)!=-1:
        input_fn_lst.append(fn)
        setnum=eval(fn.split('_')[2].lower().split('set')[-1])
        setnum_lst.append(setnum)
print(trgt)
setnum_lst

SpikeTimes.csv


[211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 226]

In [17]:
#import all files of raw data and convert it into a dictionary of spike times (may take ~1 minute)
dict_spike_times_lst=[]
for input_fn in input_fn_lst:
    dict_spike_times=dict()
    with open(input_fn) as f:
        nid=0
        for line in f:
            spikes=[]
            for val in line.split(','):
                try:
                    val=eval(val)
                    spikes.append(val)
                except SyntaxError as e:
                    pass
            dict_spike_times[nid]=spikes
            nid+=1
    dict_spike_times_lst.append(dict_spike_times)
raw=dict(zip(trialnum_lst,dict_spike_times_lst))

In [ ]:
#TODO: parse trialTimes for each set
#TODO(later): make parser for _trialData.csv into a pandas.DataFrame

In [9]:
# out_fn=f"archie_raw.json"
out_fn=f"buck_raw.json"
#save as json
save_folder=f"{nb_dir}/Data"
if not os.path.exists(save_folder):
    os.mkdir(save_folder)
os.chdir(save_folder)
out_file = open(out_fn, "w")
json.dump(raw, out_file, indent = 6)
out_file.close()

In [ ]:
#TODO: restart JVM
#TODO: import trialtimes
#TODO: get parsing working for 
#TODO: format trialtimes into labeled vector for (red/yellow/blue)

In [10]:
beep(1)

0

In [7]:
# #parse the file names
# data_dir=f"{nb_dir}/Data/gitignored"
# os.chdir(data_dir)
# fn_lst=sorted([fn for fn in os.listdir() if fn[-4:]=='.csv'])
# # print(len(fn_lst))

# #TODO: import a given csv
# fn=fn_lst[0]
# MonkName,ProjectName,StimulusSet,Subset,YYMMDD,HHMMSS,modname=fn.split('_')
# print(modname)


# setnumber_lst=[]
# for fn in fn_lst:
#     setnumber=eval(fn.split('_')[2].lower().split('set')[-1])
#     setnumber_lst.append(setnumber)
# setnumber_set=set(sorted(set(setnumber_lst)))
# #TODO: make a function mapping setnumber set to a desired .csv file name
# # setnumber=setnumber_set[0]
# fn_foo=lambda setnumber,modename:f"{MonkName}_{ProjectName}_Set{setnumber}_{YYMMDD}_{HHMMSS}_{modname}"



In [6]:
# num_neurons=obj.shape[0]


# neuron_id_lst=sorted(range(num_neurons))
# for trial_id in sorted(range(53,452)):
#     spikes_lst=[]
#     for nid in neuron_id_lst:
#         ref=obj[(nid)][0]
#         name = h5py.h5r.get_name(ref, f.id)
#         trials = np.array(f[(name)]).flatten();#print(name) #not sure if trials covers trials or neurons... its length is ~450.
#         ref=trials[trial_id]
#         name = h5py.h5r.get_name(ref, f.id)
#         spikes=np.array(f[(name)]);#print(f"{nid}:{name},")
#         spikes_lst.append(spikes)

#     dict_spike_times={}
#     for nid,spikes in zip(neuron_id_lst,spikes_lst):
#         dict_spike_times[nid]=list(spikes.flatten().astype(np.float64))

#     # the json file where the output must be stored 
# #         os.chdir(f"{nb_dir}/Data/Archie")
    